In [4]:
# First I need to categorize the data from my participants
# Trials (1-667 (might have to cut some out)), stimuli_type (Happy vs Angry), response (Approach vs Avoid),% #Congruency (congruent vs incongruent), block_type (stable vs volatile), change objectively correct to accuracy maybe and structure it as (correct vs incorrect), response_time in ms, and reward amount ( -10 or +10 cents)
# mix objectively correct and stimuli type for the Congruency condition  

#My 1st step is to create a data frame where all the participants data is stored

#Import everything 

NameError: name 'pd' is not defined

In [5]:
import pandas as pd
import matplotlib.pyplot as plt

# Now we load the data

In [15]:
import pandas as pd

data = pd.read_csv('/Volumes/project/3025011.02/raw/sub-005/ses-mri01/beh/behavioural_output_sub-005_session-01_dummy_2025-03-07_17-11-43.csv', delimiter=';')

print(data)


     trial  emotional_cue_time response objectively_correct  \
0        0        1.741364e+09       up                Even   
1        1        1.741364e+09     down                Even   
2        2        1.741364e+09      NaN                Late   
3        3        1.741364e+09       up                Even   
4        4        1.741364e+09     down                Even   
..     ...                 ...      ...                 ...   
663    663        1.741367e+09     down               False   
664    664        1.741367e+09       up                True   
665    665        1.741367e+09       up               False   
666    666        1.741367e+09       up                True   
667    667        1.741367e+09     down                True   

    subjectively_correct  response_time   RT_s  feedback_time  stimuli_type  \
0                  False   1.741364e+09  1.091   1.741364e+09             1   
1                  False   1.741364e+09  0.597   1.741364e+09             1   
2     

File exists: False


/Users/carlacoca/Documents/master_internship/experiment_output_pilots/pythonProject/3025011.02/raw/sub-005/ses-mri01/beh/behavioural_output_sub-005_session-01_dummy_2025-03-07_17-11-43.csv


FileNotFoundError: [Errno 2] No such file or directory: '/new/path/to/file.csv'